**참고문헌: 핸즈온 머신러닝(2판), 올레리앙 제롱 지음, 박해선 옮김, 11장 – 심층 신경망 훈련하기**

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/handson-ml2/blob/master/11_training_deep_neural_networks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

# 설정

먼저 몇 개의 모듈을 임포트합니다. 맷플롯립 그래프를 인라인으로 출력하도록 만들고 그림을 저장하는 함수를 준비합니다. 또한 파이썬 버전이 3.5 이상인지 확인합니다(파이썬 2.x에서도 동작하지만 곧 지원이 중단되므로 파이썬 3을 사용하는 것이 좋습니다). 사이킷런 버전이 0.20 이상인지와 텐서플로 버전이 2.0 이상인지 확인합니다.

In [ ]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## 사전 훈련된 층 재사용하기: 전이 학습(transfer learning)

일반적으로 아주 큰 규모의 DNN을 처음부터 새로 훈련하는 것은 좋은 생각이 아닙니다. 해결하려는 것과 비슷한 유형의 문제를 처리한 신경망이 이미 있는지 찾아 본 다음, 그 신경망의 하위층을 재사용하는 것이 좋을 수 있습니다. 이를 전이 학습(transfer learning)이라고 합니다. 이 방법은 훈련 속도를 크게 높일 뿐만 아니라 필요한 훈련 데이터도 크게 줄여줍니다.

동물, 식물, 자동차, 생활용품을 포함해 카테고리 100개로 구분된 이미지를 분류하도록 훈련한 DNN을 가지고 있다고 가정합시다. 그리고 이제 구체적인 자동차의 종류를 분류하는 DNN을 훈련하려고 합니다. 이런 작업들은 비슷한 점이 많고 심지어 일부 겹치기도 하므로 첫 번째 신경망의 일부를 재사용해봐야합니다. 

<figure>
<img src="https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/%EA%B7%B8%EB%A6%BC11-4%20%EC%82%AC%EC%A0%84%ED%9B%88%EB%A0%A8%EB%90%9C%20%EC%B8%B5%20%EC%9E%AC%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0.png"/>
</figure>

**NOTE:**  만약 원래 문제에서 사용한 것과 크기가 다른 이미지를 입력으로 사용한다면 원본 모델에 맞는 크키로 변경하는 전처리 단계를 추가해야합니다. 일반적으로 전이 학습은 저수준 특성이 비슷한 입력에서 잘 작동합니다.

보통 원본 모델의 출력층을 바꿔야합니다. 이 층이 새로운 작업에 가장 유용하지 않는 층이고 새로운 작업에 필요한 출력 개수와 맞지 않을 수도 있습니다. 비슷하게 원본 모델의 상위 은닉층은 하위 은닉층보다 덜 유용합니다. 새로운 작업에 유용한 고수준 특성은 원본 작업에서 유용했던 특성과는 상당히 다르기 때문입니다. 재사용할 층 개수를 잘 선정하는 것이 필요합니다.

**Tip: 작업이 비슷할수록(낮은 층부터 시작해서) 더 많은 층을 재사용하세요. 아주 비슷한 작업이라면 모든 은닉층을 유지하고 출력층만 교체합니다.**

### 케라스 모델 재사용하기

패션 MNIST 훈련 세트를 두 개로 나누어 보죠:
* `X_train_A`: 샌달과 셔츠(클래스 5와 6)을 제외한 모든 이미지
* `X_train_B`: 샌달과 셔츠 이미지 중 처음 200개만 가진 작은 훈련 세트

검증 세트와 테스트 세트도 이렇게 나눕니다. 하지만 이미지 개수는 제한하지 않습니다.

A 세트(8개의 클래스를 가진 분류 문제)에서 모델을 훈련하고 이를 재사용하여 B 세트(이진 분류)를 해결해 보겠습니다. A 작업에서 B 작업으로 약간의 지식이 전달되기를 기대합니다. 왜냐하면 A 세트의 클래스(스니커즈, 앵클 부츠, 코트, 티셔츠 등)가 B 세트에 있는 클래스(샌달과 셔츠)와 조금 비슷하기 때문입니다. 하지만 `Dense` 층을 사용하기 때문에 동일한 위치에 나타난 패턴만 재사용할 수 있습니다(반대로 합성곱 층은 훨씬 많은 정보를 전송합니다. 학습한 패턴을 이미지의 어느 위치에서나 감지할 수 있기 때문입니다. CNN 장에서 자세히 알아 보겠습니다).

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
%matplotlib inline


(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [ ]:
X_train_A.shape

(43986, 28, 28)

In [ ]:
X_train_B.shape

(200, 28, 28)

In [ ]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [ ]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.5926 - accuracy: 0.8104 - val_loss: 0.3896 - val_accuracy: 0.8662
Epoch 2/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3523 - accuracy: 0.8785 - val_loss: 0.3288 - val_accuracy: 0.8824
Epoch 3/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3171 - accuracy: 0.8896 - val_loss: 0.3014 - val_accuracy: 0.8984
Epoch 4/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2973 - accuracy: 0.8975 - val_loss: 0.2894 - val_accuracy: 0.9018
Epoch 5/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2835 - accuracy: 0.9020 - val_loss: 0.2776 - val_accuracy: 0.9066
Epoch 6/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2730 - accuracy: 0.9061 - val_loss: 0.2734 - val_accuracy: 0.9063
Epoch 7/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2641 - accuracy: 0.9092 - val_loss: 0.2716 - val_accuracy:

작업 A를 해결하는 모델을 만들고 훈련해 꽤 좋은 성능(>90% 정확도)을 얻었습니다. 이 모델을 '모델 A'로 부르겠습니다. 해당 모델을 저장하겠습니다.

In [ ]:
model_A.save("my_model_A.h5")

다음은 샌들과 셔츠 이미즈를 구분하는 작업 B를 해결하기 위해 이진 분류기를 훈련하려고 합니다(양성=셔츠, 음성=샌들). 레이블된 이미지는 겨우 200개로 준비된 데이터는 매우 적습니다. 이를 위해서 모델 A와 구조가 거의 비슷한 '모델 B'라는 새 모델을 만들어 보겠습니다. 

In [ ]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 61ms/step - loss: 0.9573 - accuracy: 0.4650 - val_loss: 0.6314 - val_accuracy: 0.6004
Epoch 2/20
7/7 [==============================] - 0s 19ms/step - loss: 0.5692 - accuracy: 0.7450 - val_loss: 0.4784 - val_accuracy: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 20ms/step - loss: 0.4503 - accuracy: 0.8650 - val_loss: 0.4102 - val_accuracy: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 20ms/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 0.3647 - val_accuracy: 0.9178
Epoch 5/20
7/7 [==============================] - 0s 31ms/step - loss: 0.3435 - accuracy: 0.9250 - val_loss: 0.3300 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 31ms/step - loss: 0.3081 - accuracy: 0.9300 - val_loss: 0.3019 - val_accuracy: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 31ms/step - loss: 0.2800 - accuracy: 0.9350 - val_loss: 0.2804 - val_accuracy: 0.9422
Epoch 8/20
7/7 [===========

모델 B는 꽤 좋은 성능을 냈습니다 (>97% 정확도). 

In [ ]:
model_B.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_20 (Dense)            (None, 300)               235500    
                                                                 
 dense_21 (Dense)            (None, 100)               30100     
                                                                 
 dense_22 (Dense)            (None, 50)                5050      
                                                                 
 dense_23 (Dense)            (None, 50)                2550      
                                                                 
 dense_24 (Dense)            (None, 50)                2550      
                                                                 
 dense_25 (Dense)            (None, 1)                

하지만 모델 B는 클래스가 두 개뿐인 매우 쉬운 문제이므로 우리는 더 좋은 성능을 내고 싶습니다. 곰곰히 생각해보면 작업 B는 모델 A가 해결하는 작업 A와 매우 비슷하다는 것을 알 수 있습니다. 혹시 전이 학습이 도움이 될 수 있는지 확인해 보겠습니다.

먼저 모델 A를 로드하고 이 모델의 층을 기반으로 새로운 모델(model_B_on_A)을 만듭니다. 출력층만 제외하고 모든 층을 재사용 하겠습니다. 

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

`model_B_on_A`와 `model_A`는 층을 공유하기 때문에 하나를 훈련하면 두 모델이 업데이트됩니다. 이를 피하려면 `model_A`를 클론한 것을 사용해 `model_B_on_A`를 만들어야 합니다. Clone_mode() 메서드로 모델 A의 구조를 복제한 후 가중치를 복사합니다(clone_model() 메서드는 가중치를 복제하지 않습니다.)

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

이제 작업 B를 위해 model_B_on_A을 훈련할 수 있습니다. 하지만 새로운 출력층이 랜덤하게 초기화되어 있으므로 큰 오차를 만들 것입니다 (적어도 처음 몇 번의 에포크 동안). 따라서 큰 오차 그레이디언트가 재사용된 가중치를 망칠 수 있습니다. 이를 피하는 한 가지 방법은 처음 몇번의 에포크 동안 재사용된 층을 동결하고 새로운 층에게 적절한 가중치를 학습할 시간을 주는 것입니다. 이를 위해 모든 층의 trainable 속성을 False로 지정하고 모델을 컴파일 합니다.

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=20,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 58ms/step - loss: 0.2643 - accuracy: 0.9400 - val_loss: 0.2789 - val_accuracy: 0.9280
Epoch 2/20
7/7 [==============================] - 0s 18ms/step - loss: 0.2547 - accuracy: 0.9400 - val_loss: 0.2694 - val_accuracy: 0.9300
Epoch 3/20
7/7 [==============================] - 0s 19ms/step - loss: 0.2454 - accuracy: 0.9400 - val_loss: 0.2607 - val_accuracy: 0.9341
Epoch 4/20
7/7 [==============================] - 0s 30ms/step - loss: 0.2369 - accuracy: 0.9400 - val_loss: 0.2526 - val_accuracy: 0.9381
Epoch 5/20
7/7 [==============================] - 0s 19ms/step - loss: 0.2289 - accuracy: 0.9450 - val_loss: 0.2451 - val_accuracy: 0.9391
Epoch 6/20
7/7 [==============================] - 0s 30ms/step - loss: 0.2216 - accuracy: 0.9450 - val_loss: 0.2379 - val_accuracy: 0.9422
Epoch 7/20
7/7 [==============================] - 0s 30ms/step - loss: 0.2146 - accuracy: 0.9500 - val_loss: 0.2313 - val_accuracy: 0.9432
Epoch 8/20
7/7 [===========

**NOTE:** 층을 동결하거나 동결을 해제한 후 반드시 모델을 컴파일 해야합니다.

In [ ]:

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 1s 46ms/step - loss: 0.1404 - accuracy: 0.9700 - val_loss: 0.1476 - val_accuracy: 0.9787
Epoch 2/16
7/7 [==============================] - 0s 30ms/step - loss: 0.1193 - accuracy: 0.9750 - val_loss: 0.1309 - val_accuracy: 0.9838
Epoch 3/16
7/7 [==============================] - 0s 19ms/step - loss: 0.1039 - accuracy: 0.9900 - val_loss: 0.1181 - val_accuracy: 0.9858
Epoch 4/16
7/7 [==============================] - 0s 31ms/step - loss: 0.0916 - accuracy: 0.9900 - val_loss: 0.1080 - val_accuracy: 0.9878
Epoch 5/16
7/7 [==============================] - 0s 20ms/step - loss: 0.0822 - accuracy: 0.9950 - val_loss: 0.1002 - val_accuracy: 0.9878
Epoch 6/16
7/7 [==============================] - 0s 30ms/step - loss: 0.0747 - accuracy: 0.9950 - val_loss: 0.0937 - val_accuracy: 0.9878
Epoch 7/16
7/7 [==============================] - 0s 31ms/step - loss: 0.0685 - accuracy: 0.9950 - val_loss: 0.0882 - val_accuracy: 0.9878
Epoch 8/16
7/7 [===========

이 모델의 테스트 정확도가 >99% 입니다. 전이 학습이 오차를 많이 줄여줬습니다.

데이터 크기-유사성에 따른 Fine-tuning 방법 (참고 블로그: https://jeinalog.tistory.com/13)

<figure>
<img src="https://raw.githubusercontent.com/Hyun-chul/DeepLearning/main/%EA%B7%B8%EB%A6%BC11-6%20transfer%20learning.png"/>
</figure>